# Westeros Tutorial - Introducing `"addon"` technologies

This tutorial shows how to establish an inter-dependency between two technologies by configuring one of them as `"addon"`  to another one, i.e., parent technology. This can be used to model additional technology features such as carbon-capture-and-storage (CCS) retrofits, passout-turbines (for optional heat cogeneration) or cooling technologies, to existing technologies.

There are several ways to tackle this issue. Lets take the example of a coal power plant (`"coal_ppl"`). All of the above mentioned additional features could be implemented by introducing different *modes* of operation for `"coal_ppl"`. For example, heat cogeneration could be implemented as a separate operation `"mode"` of `"coal_ppl"`, where instead of just generating electricity, heat can also be produced at the cost of reducing the amount of electricity generated. Another approach would make use of the generic `"relations"` in MESSAGEix, therefore linking the newly added technology representing the passout-turbine with the activity of `"coal_ppl"`. Both of these approaches have some downsides. Using a separate `"mode"` will not permit explicitly modelling investment costs and lifetime associated with the asset being added to `"coal_ppl"`. Generic relations are very flexible, but if too many of them are added, the model becomes very hard to understand. MESSAGEix offers an explicit `"addon"` formulation for tackling this issue.

The additional technology options are explicitly modelled as separate technologies, classified as `"addon"` technologies, linked to the activity of the technology to which they serve as additional configuration options, i.e., the parent technology. Through an `"addon_conversion"` factor, the activity of `"addon"` technology can further be restricted to a minimum or maximum share of the activity of the parent technology.

**Pre-requisites for running this tutorial**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (`westeros_baseline.ipynb`) and solved it successfully

In [ ]:
import ixmp

mp = ixmp.Platform()

## Making a clone of the existing scenario '*baseline*'

In [ ]:
import message_ix

model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model, "addon_technology", "illustration of addon formulation", keep_solution=False
)
scen.check_out()

### i. Setting up parameters

In [ ]:
import pandas as pd

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"
gdp_profile = pd.Series([1.0, 1.5, 1.9], index=pd.Index([700, 710, 720], name="time"))

### ii. Define helper dictionaries used for subsequent operations

In [ ]:
base_input = {
    "node_loc": country,
    "year_vtg": vintage_years,
    "year_act": act_years,
    "mode": "standard",
    "node_origin": country,
    "time": "year",
    "time_origin": "year",
}
base_output = {
    "node_loc": country,
    "year_vtg": vintage_years,
    "year_act": act_years,
    "mode": "standard",
    "node_dest": country,
    "time": "year",
    "time_dest": "year",
    "unit": "-",
}
base_capacity_factor = {
    "node_loc": country,
    "year_vtg": vintage_years,
    "year_act": act_years,
    "time": "year",
    "unit": "-",
}
base_technical_lifetime = {
    "node_loc": country,
    "year_vtg": model_horizon,
    "unit": "y",
}
base_inv_cost = {
    "node_loc": country,
    "year_vtg": model_horizon,
    "unit": "USD/kW",
}
base_fix_cost = {
    "node_loc": country,
    "year_vtg": vintage_years,
    "year_act": act_years,
    "unit": "USD/kW",
}
base_var_cost = {
    "node_loc": country,
    "year_vtg": vintage_years,
    "year_act": act_years,
    "mode": "standard",
    "time": "year",
    "unit": "USD/kWa",
}

## `"addon"` technology in MESSAGEix
This tutorial will extend the current reference-energy-system to include a demand for heat and the necessary technologies to meet this demand. Heat will be generated via a passout-turbine which will be linked to the `"coal_ppl"` using the `"addon"` formulation.

<img src='_static/addon_technologies_res.png' width='700'>

We will therefore carry out the following three steps:
1. Define a new commodity and demand for heat:
    - Define new `"commodity"` `"heat"`.
    - Parametrize `"demand"` for `"heat"`.
    
    
2. Add new technologies:
    - Add a new technology to generate heat: `"passout-turbine"`.
    - Add a new technology district heat network to transport heat to the end-use technology: `"dh_grid"`.
    - Add a new end-use technology, an in-house district heat connection which is linked to `"demand": "hs_house"`.
    
3. Link the passout-turbine to the coal_ppl using the `"addon"` feature.

### 1: Define a new commodity and demand
We therefore add a new `"commodity"` `"heat"` and a corresponding demand, which will rise at the same rate as electricity demand.

In [ ]:
# Define a new commodity `"heat"`
scen.add_set("commodity", ["heat"])

# Add heat demand at the useful level
heat_demand = pd.DataFrame(
    {
        "node": country,
        "commodity": "heat",
        "level": "useful",
        "year": [700, 710, 720],
        "time": "year",
        "value": (25 * gdp_profile).round(),
        "unit": "GWa",
    }
)
scen.add_par("demand", heat_demand)

### 2: Define new technologies

i. Heat will be generated via a pass-out turbine:
Passout-turbine (`"po_turbine"`) characteristics: The passout-turbine requires one unit of electricity to generate five units of heat. The lifetime is assumed to be 30 years, 10 years longer then that of `"coal_ppl"`.  Investment costs are 150\\$/kW compared to 500\\$/kW for `"coal_ppl"`.  A coal heatplant would have higher investment costs, approximately double that of "po_turbine". Lastly, `"po_turbine"` represents an alternative production mode of `"coal_ppl"`, hence in order to produce heat, the electricity of `"coal_ppl"` is reduced. Thus, electricity is parametrized as an input to `"po_turbine"`; for each unit of electricity, 5 units of heat can be produced. This will later also be used for establishing a "link" between `"coal_ppl"` and `"po_turbine"`.

ii. Heat will be transported via a district heating grid:
District heat (`"dh_grid"`) network characteristics: District heating networks have only very low losses as these cover only short distances (within city perimeters). We will assume the district heating network to have an efficiency of 97%.

iii. Heat demand will be linked to an end-use technology:
`"hs_house"` will represent the end-use technology, which distributes heat within the buildings.

Similar to previous tutorials, we work our way backwards, starting from the `"heat"` demand defined at the `"useful"` energy level and connecting this to the `"final"` energy level via a technology, `"hs_house"`, representing the in-house heat distribution system.

In [ ]:
from message_ix import make_df

tec = "hs_house"
scen.add_set("technology", tec)

hs_house_out = make_df(
    "output", **base_output, technology=tec, commodity="heat", level="useful", value=1.0
)
scen.add_par("output", hs_house_out)

hs_house_in = make_df(
    "input",
    **base_input,
    technology=tec,
    commodity="heat",
    level="final",
    value=1.0,
    unit="-",
)
scen.add_par("input", hs_house_in)

Next, we add the information for the district heating network.

In [ ]:
tec = "dh_grid"
scen.add_set("technology", tec)

dh_grid_out = make_df(
    "output", **base_output, technology=tec, commodity="heat", level="final", value=1.0
)
scen.add_par("output", dh_grid_out)

dh_grid_in = make_df(
    "input",
    **base_input,
    technology=tec,
    commodity="heat",
    level="secondary",
    value=1.03,
    unit="-",
)
scen.add_par("input", dh_grid_in)

Last, we add `"po_turbine"` as a technology.

In [ ]:
tec = "po_turbine"
scen.add_set("technology", tec)

po_out = make_df(
    "output",
    **base_output,
    technology=tec,
    commodity="heat",
    level="secondary",
    value=1.0,
)
scen.add_par("output", po_out)

po_in = make_df(
    "input",
    **base_input,
    technology=tec,
    commodity="electricity",
    level="secondary",
    value=0.2,
    unit="-",
)
scen.add_par("input", po_in)

po_tl = make_df(
    "technical_lifetime", **base_technical_lifetime, technology=tec, value=30
)
scen.add_par("technical_lifetime", po_tl)

po_inv = make_df("inv_cost", **base_inv_cost, technology=tec, value=150)
scen.add_par("inv_cost", po_inv)

po_fix = make_df("fix_cost", **base_fix_cost, technology=tec, value=15)
scen.add_par("fix_cost", po_fix)

### 3: Link `"po_turbine"` with `"coal_ppl"`
`"po_turbine"` could already operate as all required parameters are defined, yet without a link to the activity of `"coal_ppl"`, `"po_turbine"` has the possibility of using electricity generated from either `"coal_ppl"` or `"wind_ppl"`.  But because `"po_turbine"` is an addon component to `"coal_ppl"` a distinct linkage needs to be established.

First, the newly added technology `"po_turbine"` needs to be classified as an `"addon"` technology

In [ ]:
scen.add_set("addon", "po_turbine")

Next, we need a new `"type_addon"`, which we will name `"cogeneration_heat"`.  We will classify the `"po_turbine"` via the *category* `"addon"` as one of the addon technologies as part of this specific `"type_addon"`.  In some cases, for example when modelling cooling technologies, multiple technologies can be classfied within a single `"type_addon"`.

Via the set `"map_tec_addon"` we map the electricity generation technology, `"coal_ppl"`, to the `"addon"` technology, `"po_turbine"`. Multiple technologies, for example further fossil powerplants, could also be added to this `"type_addon"` so as to be able to produce heat via `"po_turbine"`.

Note: the `"addon"` technology as well as the parent technology must have the same `"mode"`.

In [ ]:
type_addon_ch = "cogeneration_heat"
addon_tech = "po_turbine"
tec = "coal_ppl"
scen.add_cat("addon", type_addon_ch, addon_tech)
scen.add_set(
    "map_tec_addon", pd.DataFrame({"technology": tec, "type_addon": [type_addon_ch]})
)

The last step required in order to link the `"coal_ppl"` is to define the `"addon_conversion"` factor between the `"coal_ppl"` and the `"type_addon"`.  This is important, because the `"coal_ppl"` generates electricity while the `"po_turbine"` generates heat. Therefore, we can use the inverse of the `"input"` coefficient from the `"po_turbine"`.

In [ ]:
df = pd.DataFrame(
    {
        "node": country,
        "technology": tec,
        "year_vtg": vintage_years,
        "year_act": act_years,
        "mode": "standard",
        "time": "year",
        "type_addon": type_addon_ch,
        "value": 5,
        "unit": "-",
    }
)
scen.add_par("addon_conversion", df)

Although not necessary for this specific example, it is also possible to limit the activity of `"po_turbine"` to a specific share of `"coal_ppl"` activity. In the example below, `"po_turbine"` is limited to using 15% of `"coal_ppl"` activity.  Likewise, a constraint on the minimum amount of electricity used from `"po_turbine"` can be applied by using the parameter `"addon_lo"`.

In [ ]:
# Index for "addon_up" is ["node", "technology", "year_act",
#                          "mode", "time", "type_addon",
#                          "value", "unit"]
df = pd.DataFrame(
    {
        "node": country,
        "technology": tec,
        "year_act": act_years,
        "mode": "standard",
        "time": "year",
        "type_addon": type_addon_ch,
        "value": 0.15,
        "unit": "-",
    }
)
scen.add_par("addon_up", df)

### Commit and solve

In [ ]:
scen.commit(comment="define parameters for renewable implementation")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

# Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

# Create a Reporter object to describe and carry out reporting
# calculations and operations (like plotting) based on `base` and `scen`
rep_bl = Reporter.from_scenario(base)
rep_addon = Reporter.from_scenario(scen)

# "prepare_plots" enables several to describe reporting operations, e.g.
# "plot activity", "plot capacity", or "plot prices"
# See message_ix/util/tutorial.py for more information
prepare_plots(rep_bl)
prepare_plots(rep_addon)

## Activity
***

### Scenario: '*baseline*'

In [ ]:
rep_bl.set_filters(t=["coal_ppl", "wind_ppl"])
rep_bl.get("plot activity")

### Scenario: '*addon_technology*'

In [ ]:
rep_addon.set_filters(t=["coal_ppl", "wind_ppl"])
rep_addon.get("plot activity")

In [ ]:
rep_addon.set_filters(t=["po_turbine"])
rep_addon.get("plot activity")

### Question
Comparing the electricity generation of wind power plants in *baseline* and in this scenario shows that wind is generating more electricity now. Can you explain the reason? You can find the answer at the end of this tutorial.

## Capacity
***
The behavior observed for the activity of the two electricity generation technologies is reflected in the capacity.

### Scenario: '*baseline*'

In [ ]:
rep_bl.set_filters(t=["coal_ppl", "wind_ppl"])
rep_bl.get("plot capacity")

### Scenario: '*addon_technology*'

In [ ]:
rep_addon.set_filters(t=["coal_ppl", "wind_ppl"])
rep_addon.get("plot capacity")

## Prices
***
The resulting impact on the electricity price is negligable, though. Yet we can see that the price of heat is significantly lower than that of light.

### Scenario: '*baseline*'

In [ ]:
rep_bl.set_filters(c=["light"])
rep_bl.get("plot prices")

### Scenario: '*addon_technology*'

In [ ]:
rep_addon.set_filters(c=["light"])
rep_addon.get("plot prices")

In [ ]:
rep_addon.set_filters(c=["heat"])
rep_addon.get("plot prices")

### Answer to the question:
In the new scenario ('*addon_technology*'), the effects of the addon technology can be seen when comparing the activity to the baseline scenario ('*baseline*'). From 700 onwards, the activity of the `"wind_ppl"` has increased to compensate for the electricity required from the `"coal_ppl"` for use in the `"po_turbine"`. In 720, when the `"wind_ppl"` is phased out, more electricity is required to be produced by the `"coal_ppl"`. 

In [ ]:
mp.close_db()